In [4]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
import os

def formatear_numero_celular(numero):
    """
    Formatea el número de celular agregando +591 si no lo tiene
    """
    # Limpiar el número de espacios y caracteres especiales
    numero_limpio = ''.join(c for c in numero if c.isdigit())
    
    # Si el número ya empieza con 591, agregar solo el +
    if numero_limpio.startswith('591'):
        return f"+ {numero_limpio}"
    
    # Si no tiene 591, agregarlo con el +
    return f"+591{numero_limpio}"
def leer_datos_archivo(nombre_archivo):
    """
    Lee el archivo de datos y extrae nombres y números de celular
    Formato esperado: celular:campo2:nombre:apellido
    """
    personas = []
    
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
            for i, linea in enumerate(archivo, 1):
                linea = linea.strip()
                if linea:  # Si la línea no está vacía
                    # Separar por dos puntos (:)
                    partes = linea.split(':')
                    
                    if len(partes) >= 4:
                        # Formato: celular:campo2:nombre:apellido
                        celular_original = partes[0].strip()
                        celular_formateado = formatear_numero_celular(celular_original)
                        nombre = partes[2].strip()
                        apellido = partes[3].strip()
                        nombre_completo = f"{nombre} {apellido}".strip()
                        
                        personas.append({
                            'nombre': nombre_completo,
                            'nombre_individual': nombre,
                            'apellido': apellido,
                            'celular': celular_formateado,
                            'celular_original': celular_original,
                            'linea': i
                        })
                    elif len(partes) >= 2:
                        # Formato alternativo con menos campos
                        celular_original = partes[0].strip()
                        celular_formateado = formatear_numero_celular(celular_original)
                        nombre = ':'.join(partes[1:]).strip()  # Unir el resto como nombre
                        
                        personas.append({
                            'nombre': nombre,
                            'nombre_individual': nombre.split()[0] if nombre.split() else nombre,
                            'apellido': ' '.join(nombre.split()[1:]) if len(nombre.split()) > 1 else '',
                            'celular': celular_formateado,
                            'celular_original': celular_original,
                            'linea': i
                        })
                    else:
                        print(f"⚠️  Línea {i} con formato incorrecto: {linea}")
    
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo '{nombre_archivo}'")
        print("   Asegúrate de que el archivo esté en el mismo directorio que este script.")
        return []
    except Exception as e:
        print(f"❌ Error al leer el archivo: {e}")
        return []
    
    return personas

def crear_carta(persona, numero_carta, ruta_logo=None):
    """
    Crea un documento Word con la carta personalizada
    ruta_logo: ruta al archivo de imagen del logo (opcional)
    """
    # Crear nuevo documento
    doc = Document()
    
    # Configurar márgenes
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1.5)
        section.right_margin = Inches(1.5)
    
    # Agregar logo si se proporciona la ruta
    if ruta_logo and os.path.exists(ruta_logo):
        try:
            logo_paragraph = doc.add_paragraph()
            logo_paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
            
            # Agregar la imagen del logo
            run = logo_paragraph.runs[0] if logo_paragraph.runs else logo_paragraph.add_run()
            
            # Ajustar el tamaño del logo (puedes modificar estos valores)
            logo_width = Inches(2)  # Ancho del logo
            logo_height = Inches(0.6)  # Alto del logo
            
            # Insertar imagen
            logo_paragraph.add_run().add_picture(ruta_logo, width=logo_width, height=logo_height)
            
            
        except Exception as e:
            print(f"⚠️  No se pudo cargar el logo: {e}")
            print("   La carta se generará sin logo.")
    
    # Fecha actual
    fecha_actual = datetime.now().strftime("%d de %B de %Y")
    meses = {
        'January': 'enero', 'February': 'febrero', 'March': 'marzo',
        'April': 'abril', 'May': 'mayo', 'June': 'junio',
        'July': 'julio', 'August': 'agosto', 'September': 'septiembre',
        'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
    }
    for eng, esp in meses.items():
        fecha_actual = fecha_actual.replace(eng, esp)
    # Agregar "La Paz, " antes de la fecha
    fecha_completa = f"La Paz, {fecha_actual}"

    # Encabezado con fecha
    fecha_p = doc.add_paragraph()
    fecha_p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    fecha_run = fecha_p.add_run(fecha_completa)
    fecha_run.italic = True
    
    # Espacio
    doc.add_paragraph()
    
    # Título de la carta
    #titulo = doc.add_heading(f'Carta de Contacto Personalizado #{numero_carta}', level=1)
    #titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Espacio
    doc.add_paragraph()
    
    # Saludo personalizado
    # Párrafo 1: "Estimado/a:"
    p1 = doc.add_paragraph()
    p1.add_run('Estimado/a:').bold = True
    # Ajusta el espaciado posterior
    p1.paragraph_format.space_after = Pt(0)  # Cambiado a 0 para eliminar espacio

    # Párrafo 2: Nombre de la persona
    p2 = doc.add_paragraph()
    p2.add_run(f"{persona['nombre']}").bold = True
    # Ajusta el espaciado posterior
    p2.paragraph_format.space_after = Pt(0)  # Cambiado a 0 para eliminar espacio

    # Párrafo 3: "Presente.-"
    p3 = doc.add_paragraph()
    presente = p3.add_run('Presente.-')
    presente.bold = True
    presente.underline = True
    presente.italic = True
    doc.add_paragraph()  # Espacio adicional
    
    # Asunto
    asunto = "Asunto: Confirmación de información telefónica"  # Puedes cambiar este texto
    asunto_p = doc.add_paragraph()
    asunto_p.alignment = WD_ALIGN_PARAGRAPH.LEFT
    asunto_run = asunto_p.add_run(asunto)
    asunto_run.bold = True

    # Cuerpo de la carta
    parrafos = [
        "Espero que esta carta le encuentre en perfecto estado de salud y bienestar. Me dirijo a usted de manera cordial para establecer contacto y confirmar la información que tenemos registrada en nuestros sistemas.",
        
        f"Según nuestros registros, su número de contacto celular es: {persona['celular']}",
        
        "Le escribimos para verificar que esta información sea correcta y actualizada. En caso de que haya algún cambio o corrección que deba realizarse, le agradecería mucho que se pusiera en contacto con nosotros a la brevedad posible.",
        
        "Su comunicación es muy importante para nosotros, y queremos asegurarnos de mantener todos los datos de contacto al día para poder brindarle el mejor servicio posible.",
        
        "Agradezco de antemano su tiempo y atención a esta correspondencia. Quedamos a la espera de su pronta respuesta.",
        
        "Sin más por el momento, le envío un cordial saludo."
    ]
    
    for i, parrafo_texto in enumerate(parrafos):
        p = doc.add_paragraph(parrafo_texto)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        
        # Resaltar el número de celular
        if i == 1:  # Segundo párrafo que contiene el celular
            for run in p.runs:
                if persona['celular'] in run.text:
                    run.bold = True
    
    # Espacios antes de la firma
    doc.add_paragraph()
    
    # Firma
    firma_p = doc.add_paragraph()
    firma_p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    
    firma_p.add_run("Atentamente,").bold = True
    firma_p.add_run("\n")

    # Añadir imagen de firma
    try:
        ruta_firma = "Parcial1_Candia_Zarco_Valeria_Ej2_img2.png"  # Cambia esta ruta por la ubicación de tu imagen
        firma_p = doc.add_paragraph()
        firma_p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    
        # Insertar la imagen con tamaño específico (opcional)
        run_firma = firma_p.add_run()
        run_firma.add_picture(ruta_firma, width=Inches(1.3), height=Inches(0.5))  # Ajusta el tamaño según necesites
    
    except Exception as e:
        print(f"Error al cargar la imagen: {e}")
        # Si no se puede cargar la imagen, añadir espacio para firma manual
        for i in range(3):
            doc.add_paragraph()
    
    firma_p.add_run("\n")
    firma_p.add_run("Candia Zarco Valeria Avril").bold = True
    firma_p.add_run("\n")
    firma_p.add_run("Directora de asuntos estudiantiles").bold = False
    firma_p.add_run("\n")
    firma_p.add_run("Decanatura de Admisiones y Asuntos Estudianties - DAAE").italic = True
    
    return doc

def generar_cartas_masivas():
    """
    Función principal que genera todas las cartas
    """
    print("🔄 Iniciando generación de cartas personalizadas...")
    print("=" * 50)
    
    # Configuración del logo
    RUTA_LOGO = "Parcial1_Candia_Zarco_Valeria_Ej2_img1.png"  # ← CAMBIA AQUÍ EL NOMBRE DE TU ARCHIVO DE LOGO
    
    # Verificar si existe el logo
    if os.path.exists(RUTA_LOGO):
        print(f"✅ Logo encontrado: {RUTA_LOGO}")
    else:
        print(f"⚠️  Logo no encontrado en: {RUTA_LOGO}")
        print("   Las cartas se generarán sin logo.")
        print("   Para agregar logo, coloca tu imagen (PNG, JPG, etc.) y actualiza RUTA_LOGO")
    
    # Leer datos del archivo
    personas = leer_datos_archivo('datos_cell_nombres.txt')
    
    if not personas:
        print("❌ No se pudieron cargar los datos. Proceso terminado.")
        return
    
    print(f"✅ Se cargaron {len(personas)} registros del archivo.")
    print("📋 Datos cargados (primeros 5 registros):")
    for i, persona in enumerate(personas[:5], 1):
        print(f"   {i}. {persona['nombre']} - Cel: {persona['celular']}")
    if len(personas) > 5:
        print(f"   ... y {len(personas) - 5} más")
    
    print("\n🔄 Generando cartas en formato DOCX...")
    
    # Crear carpeta para las cartas si no existe
    carpeta_cartas = "cartas_generadas"
    if not os.path.exists(carpeta_cartas):
        os.makedirs(carpeta_cartas)
    
    # Generar cantidad específica de cartas para pruebas
    # Cambia el número 3 por la cantidad que desees generar
    CANTIDAD_PRUEBAS = 10  # ← MODIFICA ESTE NÚMERO PARA PRUEBAS
    cantidad_cartas = min(CANTIDAD_PRUEBAS, len(personas))
    cartas_generadas = []
    
    for i in range(cantidad_cartas):
        persona = personas[i]
        numero_carta = i + 1
        
        try:
            # Crear la carta (con o sin logo)
            ruta_logo = RUTA_LOGO if os.path.exists(RUTA_LOGO) else None
            doc = crear_carta(persona, numero_carta, ruta_logo)
            
            # Nombre del archivo usando nombre y apellido
            nombre_archivo_limpio = f"{persona['nombre_individual']}_{persona['apellido']}".replace(" ", "_")
            nombre_archivo_limpio = ''.join(c for c in nombre_archivo_limpio if c.isalnum() or c in ('_', '-')).strip('_')
            nombre_archivo = f"carta_{numero_carta:02d}_{nombre_archivo_limpio}.docx"
            ruta_archivo = os.path.join(carpeta_cartas, nombre_archivo)
            
            # Guardar el documento
            doc.save(ruta_archivo)
            
            cartas_generadas.append({
                'numero': numero_carta,
                'nombre': persona['nombre'],
                'celular': persona['celular'],
                'archivo': nombre_archivo
            })
            
            print(f"✅ Carta {numero_carta:2d}/{CANTIDAD_PRUEBAS} generada: {persona['nombre']}")
            
        except Exception as e:
            print(f"❌ Error al generar carta para {persona['nombre']}: {e}")
    
    # Resumen final
    print("\n" + "=" * 50)
    print("🎉 GENERACIÓN COMPLETADA")
    print(f"📊 Total de cartas generadas: {len(cartas_generadas)}")
    print(f"📁 Ubicación: ./{carpeta_cartas}/")
    print("\n📋 Resumen de cartas generadas:")
    
    for carta in cartas_generadas:
        print(f"   • {carta['archivo']}")
        print(f"     Para: {carta['nombre']} (Cel: {carta['celular']})")
    
    print(f"\n✨ Las cartas están listas para usar en formato Word (.docx)")

# Ejecutar el generador
if __name__ == "__main__":
    try:
        generar_cartas_masivas()
    except KeyboardInterrupt:
        print("\n\n⚠️  Proceso interrumpido por el usuario.")
    except Exception as e:
        print(f"\n❌ Error inesperado: {e}")
    finally:
        print("\n👋 Presiona Enter para salir...")
        input()

🔄 Iniciando generación de cartas personalizadas...
✅ Logo encontrado: Parcial1_Candia_Zarco_Valeria_Ej2_img1.png
✅ Se cargaron 500 registros del archivo.
📋 Datos cargados (primeros 5 registros):
   1. Celia Mamani - Cel: + 59173299643
   2. Victor Velasco - Cel: + 59173299642
   3. Miriam Ortiz de Pérez - Cel: + 59173299640
   4. Leo Vacadias - Cel: + 59173299637
   5. Fer Alanoca - Cel: + 59173299636
   ... y 495 más

🔄 Generando cartas en formato DOCX...
✅ Carta  1/10 generada: Celia Mamani
✅ Carta  2/10 generada: Victor Velasco
✅ Carta  3/10 generada: Miriam Ortiz de Pérez
✅ Carta  4/10 generada: Leo Vacadias
✅ Carta  5/10 generada: Fer Alanoca
✅ Carta  6/10 generada: Carmen Mendoza
✅ Carta  7/10 generada: Abram Ticona Ayala
✅ Carta  8/10 generada: Sabina Delgado
✅ Carta  9/10 generada: Fernanda Qm
✅ Carta 10/10 generada: Luis Beymar

🎉 GENERACIÓN COMPLETADA
📊 Total de cartas generadas: 10
📁 Ubicación: ./cartas_generadas/

📋 Resumen de cartas generadas:
   • carta_01_Celia_Mamani.do